<a href="https://colab.research.google.com/github/SeekingNirvaana/LSTM_Projects/blob/main/Text_Generation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Generator Using Keras (LSTM in RNN)

## Importing necessary libraries

In [9]:
import numpy as np
import pandas as pd
import re
import tensorflow
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
# from keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers import Adam, RMSprop # - Works
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [24]:
working_on_colab = False
if working_on_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import os

    dir_to_drive_Colab = "/content/drive/MyDrive/DATASETS/"
    os.listdir(dir_to_drive_Colab)

In [25]:
os.system("mkdir ./data/")
if working_on_colab:
    os.system(f"cp -r {dir_to_drive_Colab}data/* ./data/")

In [11]:
token_type = 'word'

In [12]:
# Step 1. Loading the text and performing some cleanup!
# o data file contaning "Alice in Wonderland"

filename = "/content/drive/MyDrive/DATASETS/data.txt"

# opening the file
with open(filename, encoding='utf-8-sig') as f:
    text = f.read()


# Step 2. Removing text before and after the main stories 
start = text.find("CHAPTER I\n\n                      Down the Rabbit-Hole\n\n\n")
end = text.find("\n\n                             THE END")
text = text[start:end]

# Printing out the result:
print(len(text))

148329


## Decide if we are training the model from scratch and/or loading the pre-trained model

In [17]:
load_saved_model = False
train_model = True

## Cleaning up

In [13]:
# Step 3. Separating every chapter:
seq_length = 20 
start_story = '~ ' * seq_length
# adding the first chapter name back:
text = start_story + text

# Step 4. lowering the case
text = text.lower()
# separating every chapter with distinguished symbols for a better training
text = text.replace('\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)


## Tokenizing text
Splitting up the text into individual pieces, such as words/characters. Tokenization helps to "label" each individual unit of text. So, the labeled text can be used for supervised learning.

In [14]:
if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

# printing interesting quantities:
print(f"Number of tokenized words: {total_words}")
# Mapping the dictionary between words and indices
print(tokenizer.word_index)
# text after tokenization
print(token_list)

Number of tokenized words: 2656
{',': 1, '.': 2, 'the': 3, '`': 4, "'": 5, 'and': 6, 'to': 7, '-': 8, 'a': 9, 'she': 10, 'it': 11, 'of': 12, 'said': 13, '!': 14, 'i': 15, 'alice': 16, 'in': 17, 'you': 18, 'was': 19, 'that': 20, 'as': 21, '~': 22, 'her': 23, ':': 24, 'at': 25, '?': 26, ';': 27, 'on': 28, 'all': 29, 'with': 30, 'had': 31, 'but': 32, 'for': 33, 'so': 34, 'be': 35, 'very': 36, 'not': 37, 'what': 38, 'this': 39, 'they': 40, 'little': 41, 'he': 42, 'out': 43, '"': 44, 'is': 45, 'one': 46, 'down': 47, 'up': 48, 'if': 49, 'his': 50, 'then': 51, 'about': 52, 'no': 53, 'them': 54, 'know': 55, 'like': 56, 'were': 57, 'would': 58, 'went': 59, 'again': 60, 'herself': 61, 'do': 62, 'have': 63, 'when': 64, 'or': 65, 'could': 66, 'there': 67, 'thought': 68, 'off': 69, 'time': 70, 'how': 71, 'queen': 72, 'into': 73, 'see': 74, 'me': 75, 'well': 76, 'did': 77, 'your': 78, 'who': 79, "don't": 80, 'king': 81, 'now': 82, '*': 83, "i'm": 84, 'by': 85, 'began': 86, 'my': 87, 'its': 88, 'an':



### LSTM network will be trained to predict the next word in a sequence, given a sequence of words preceding this point.

### The parameter of our training process (X) is a sequence length which we use for model training

### While, the response variable (Y) for each sequence is the subsequent/next word. 

### Using one-hot encoding into a vector of length 2656 (the number of distinct words in the vocabulary)

In [15]:
def generate_sequences(token_list, step):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    
    # one-hot encoding, creating a categorical variable:
    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

step = 1
seq_length = 20

X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

# printing output:
print(f"Inout shape: {X.shape}")
print(f"Output shape: {y.shape}")


Number of sequences: 35564 

Inout shape: (35564, 20)
Output shape: (35564, 2656)


## Building LSTM model

In [18]:
if load_saved_model:
    model = load_model('./saved_models/lr=0.001.h5')

else:

    n_units = 256
    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    x = LSTM(n_units)(x)
    # x = Dropout(0.2)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)
    learning_rate = 0.001
    opti = RMSprop(learning_rate = learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opti)

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


## Fitting the model

In [21]:
epochs = 100
batch_size = 32
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)

Epoch 1/100
1112/1112 [==============================] - 140s 123ms/step - loss: 5.3529
Epoch 2/100
1112/1112 [==============================] - 124s 111ms/step - loss: 4.8387
Epoch 3/100
1112/1112 [==============================] - 118s 106ms/step - loss: 4.6452
Epoch 4/100
1112/1112 [==============================] - 117s 105ms/step - loss: 4.4996
Epoch 5/100
1112/1112 [==============================] - 115s 104ms/step - loss: 4.3841
Epoch 6/100
1112/1112 [==============================] - 118s 106ms/step - loss: 4.2748
Epoch 7/100
1112/1112 [==============================] - 116s 104ms/step - loss: 4.1909
Epoch 8/100
1112/1112 [==============================] - 116s 105ms/step - loss: 4.1127
Epoch 9/100
1112/1112 [==============================] - 113s 102ms/step - loss: 4.0076
Epoch 10/100
1112/1112 [==============================] - 114s 102ms/step - loss: 3.9190
Epoch 11/100
1112/1112 [==============================] - 112s 100ms/step - loss: 3.8861
Epoch 12/100
1112/1112 [======

In [26]:
# Saving the model, in h5 format
import os
os.system("mkdir ./saved_models/")

# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save(f'./saved_models/lr={learning_rate}.h5')

if working_on_colab:
    os.system(f"cp -r ./saved_models/lr={learning_rate}.h5 {dir_to_drive_Colab}/")


# Saving the model, in TensorFlow format directly
os.system("mkdir ./saved_models/")
# saving the model in tensorflow format
model.save(f'./lr={learning_rate}_tf', save_format='tf')
if working_on_colab:
    os.system(f"cp -r ./lr={learning_rate}_tf {dir_to_drive_Colab}/")

# loading the saved model
loaded_model = load_model(f'./lr={learning_rate}_tf')

# retraining the model if needed, etc. ...
loaded_model.summary()

INFO:tensorflow:Assets written to: ./lr=0.001_tf/assets


INFO:tensorflow:Assets written to: ./lr=0.001_tf/assets


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


In [27]:
# creating the folder called saved_models
os.system("mkdir ./saved_models/")

# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
# saving the model in .h5 format
model.save(f'./saved_models/lr={learning_rate}.h5')

# loading this model:
new_model = load_model(f'./saved_models/lr={learning_rate}.h5')
# checking the summary:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         265600    
                                                                 
 lstm (LSTM)                 (None, 256)               365568    
                                                                 
 dense (Dense)               (None, 2656)              682592    
                                                                 
Total params: 1,313,760
Trainable params: 1,313,760
Non-trainable params: 0
_________________________________________________________________


# Generating text

In [28]:
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "~":
            break
            
        if token_type == 'word':
            output_text += output_word + " "
            seed_text += output_word + " "
        else:
            output_text += output_word + " " 
            seed_text += output_word + " "
            
    return output_text


In [29]:
seed_text = ""
gen_words = 50

print('Temp 0.2')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.2))
print('Temp 0.33')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.33))
print('Temp 0.5')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 0.5))

Temp 0.2
. chapter . the didn't see how to see she was here about this - tone was long ; and the great little was in the same tone , and she thought it to be came ! . ` it as great long ! . but you perhaps they don't 
Temp 0.33
. chapter the very queen . so very long and she began be a gryphon in a large queen found this way do get they out of . i replied , if you . ' . ` i don't tell off , ' the mock turtle replied : . ` 
Temp 0.5
. chapter hatter . caterpillar . ` and are me ! ' he out . ` yet , so he should ! ' the hatter went on in a - out . ` which have never to do that you ? ' so we then looked at it . ` 


In [30]:
print('Temp 1.0')
print (generate_text(seed_text, gen_words, model, seq_length, temp = 1))

Temp 1.0
. chapter his from the mock turtle to see a other know they called . as they sat getting off however it . ) . ` come on into a your , ' said the gryphon , ` i know ' . ) . ` that's the right talking , 
